#### [Single- and Multi-process Data Loading](https://pytorch.org/docs/stable/data.html#single-and-multi-process-data-loading)
- DataLoader는 싱글 프로세스를 데이터 로딩시 디폴트로 사용한다.
- 파이썬 프로세스 내에서,  [Global Interpreter Lock (GIL)](https://wiki.python.org/moin/GlobalInterpreterLock)은 스레드 간의 파이썬 코드가 완전히 병렬화 되는 것을 막는다.
- 데이터 로딩시 계산 코드 수행 막는 것을 방지 하기 위해서(*), 파이토치는 num_workers 인자를 도입하여 다중 프로세스 데이터 로딩을 수행할 수 있게 한다.  
  
(*) '프로세스 수행 중 데이터 로딩이 병목의 원인이 되지 않기 위해서'로 이해하였다

##### Single-process data loading (default) (num_worker=0)
- 해당 모드에서는 DataLoader가 초기화 되었던 프로세스와 동일 프로세스에서 데이터 Fetching을 수행한다. 따라서, 데이터 로딩이 Computing을 막을 수도 있다(*). 그러나 해당 모드는 프로세스들 간의 데이터를 공유하는 경우에 자원(e.g. Shared Memory, File Discriptor)이 제한되어 있을 때 선호될 수 있다.  
- 혹은, 전체 데이터셋 사이즈가 작고, 메모리에 전부 로드 될 수 있을때 선호될 수 있다.
- 또한, Single-process 로딩은 가독성이 상대적으로 나은 에러를 보이기 때문에 디버깅 하기에 편리하다

(*) 왜냐하면, 데이터로딩과 그 외 작업을 수행하는 프로세스를 다르게 두면 병렬로 수행할 수 있으나, 로딩과 그 외 작업을 하나의 프로세스에서 수행하면 데이터 로딩이 병목을 일으킬 수 있기 때문.

##### Multi-process data loading

- num_workers 인수를 양수로 두면, 인수로 지정한 갯수 만큼의 다중 프로세스 데이터로딩을 수행한다.  
- 해당 모드에서 DataLoader 반복자가 생성될 때 마다 (즉, enumerate(dataloader)을 호출할 때 마다) num_workers 갯수 만큼의 worker 프로세스 들이 생성된다 (반복자를 호출할 때 마다 프로세스가 생성되어 데이터를 가져온다??!!)
- 이때, dataset, collate_fn, worker_init_fn 인자가 각 worker에 넘겨져서, 데이터를 fetch 하고 초기화를 수행한다.  
- 이 말인 즉슨 내부 IO로 각 프로세스가 데이터 접근을 동시에 하여, collate_fn 함수로 변환을 수행한다는 얘기.

Warning  
- 몇몇 반복이 있은 후에 *로더 worker 프로세스*는, *worker 프로세스에서 접근하는 부모 프로세스의 모든 파이썬 객체에 대해* 부모 프로세스와 동일한 양의 CPU 메모리를 소비한다(*). 
- 이는 데이터가 많이 크거나, 많은 worker가 있는 경우(전체 메모리 사용량 : number of workers * 부모 프로세스 사이즈 크기) 문제가 될 수 있다. 
- 위의 문제에 대한 간단한 해결책은 파이썬 객체를 Pandas, Numpy, PyArror 객체와 같이 non-refcounted 표현으로 바꾸는 것이다([관련 이슈](https://github.com/pytorch/pytorch/issues/13246#issuecomment-905703662))


(*) 자식 로더 프로세스도 부모 프로세스 만큼 CPU를 소모한다(?) 그래서, '전체 메모리 사용량 = number of workers x 부모 프로세스 사이즈 크기'로 표현한 것 같다.

- [torch.utils.data.get_worker_info()](https://pytorch.org/docs/stable/data.html#torch.utils.data.get_worker_info)는 worker 프로세스의 아래의 정보를 제공한다.
  - worker id
  - (가져오는) 데이터 복사본
  - 초기 seed   
- 메인 프로세스에서는 None을 반환한다.
- 사용자는 데이터 코드에서 worker_init_fn를 사용하여 데이터 셋 복사본을 개별 구성하고, 코드가 worker 프로세스에서 실행중인지 여부를 파악할 수 있다. 특히나 데이터를 공유할 때 유용하다.

- 맵 스타일 데이터 셋이서는 메인 프로세스가 *sampler*를 이용해서 인덱스를 생성하고, workers에 보낸다. 따라서, 모든 무작위 셔플은 메인 프로세스에서 실행된다.
- 반복자 스타일 데이터 셋이서는 각 worker 프로세스가 dataset 객체 복사본을 얻기 때문에, naive 한 다중 프로세스 로딩은 데이터 중복을 일으킬 수 있다.
- [torch.utils.data.get_worker_info()](https://pytorch.org/docs/stable/data.html#torch.utils.data.get_worker_info)와 worker_init_fn을 이용하여 각 복사본을 독립적으로 구성할 수 있다. 

- 일반적으로 다중 프로세스 로딩에서 CUDA 텐서를 반환하는 것을 권장하진 않는다[CUDA Mutiprocessing](https://pytorch.org/docs/stable/notes/multiprocessing.html#multiprocessing-cuda-note)
- 대신에, (자동 메모리 고정)[https://pytorch.org/docs/stable/data.html#memory-pinning] (pin_memory=True 설정)을 사용하여 GPU로 데이터 전송을 빠르게 하기 권장한다.